In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# import matplotlib to show plots inline.
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import glob
import os

import astropy.units as u
import matplotlib as mpl
import sys 
sys.path.append("../../")
from plot_OpSims import get_metric_medians
from script_utils import get_opsim_areas

In [3]:
# import maf python modules
import lsst.sims.maf.db as db
import lsst.sims.maf.metrics as metrics
import lsst.sims.maf.slicers as slicers
import lsst.sims.maf.stackers as stackers
import lsst.sims.maf.plots as plots
import lsst.sims.maf.metricBundles as metricBundles

In [4]:
# import convenience functions
import sys 
sys.path.append("../../../LSST_OpSim")
from Scripts_NBs.opsimUtils import *

## Load the results
We first load the results of the run into memory. We will read jointly the FBS v1.5 and FBS v1.6 results, and will differentiate at the time of plotting.

In [5]:
NSIDE = 64
your_username = "rjassef"
folder_mafoutput = "Nqso_EM5_{0:d}".format(NSIDE)
resultDbPath = '/home/idies/workspace/Temporary/{0}/scratch/MAFOutput/{1}'.format(
    your_username, folder_mafoutput)
metricDataPath = '/home/idies/workspace/Temporary/{0}/scratch/MAFOutput/{1}/MetricData/'.format(
    your_username, folder_mafoutput)

In [6]:
# get a dictionary of resultDb from given directory
resultDbs = getResultsDbs(resultDbPath)

In [7]:
# retrieve metricBundles for each opsim run and store them in a dictionary
bundleDicts_raw = dict()
for runName in resultDbs:
    bundleDicts_raw[runName] = bundleDictFromDisk(resultDbs[runName], runName, metricDataPath)

Healpix slicer using NSIDE=64, approximate resolution 54.967783 arcminutes
Healpix slicer using NSIDE=64, approximate resolution 54.967783 arcminutes
Healpix slicer using NSIDE=64, approximate resolution 54.967783 arcminutes
Healpix slicer using NSIDE=64, approximate resolution 54.967783 arcminutes
Healpix slicer using NSIDE=64, approximate resolution 54.967783 arcminutes
Healpix slicer using NSIDE=64, approximate resolution 54.967783 arcminutes
Healpix slicer using NSIDE=64, approximate resolution 54.967783 arcminutes
Healpix slicer using NSIDE=64, approximate resolution 54.967783 arcminutes
Healpix slicer using NSIDE=64, approximate resolution 54.967783 arcminutes
Healpix slicer using NSIDE=64, approximate resolution 54.967783 arcminutes
Healpix slicer using NSIDE=64, approximate resolution 54.967783 arcminutes
Healpix slicer using NSIDE=64, approximate resolution 54.967783 arcminutes
Healpix slicer using NSIDE=64, approximate resolution 54.967783 arcminutes
Healpix slicer using NSID

In [8]:
#Rearrange the bundleDicts_raw dictionary so that the keys are always the same as for the first run.
dbRuns = list(bundleDicts_raw.keys())
Keys = list(bundleDicts_raw[dbRuns[0]].keys())
bundleDicts = dict()
for runName in dbRuns:
    bundleDicts[runName] = dict()
    Keys_raw = bundleDicts_raw[runName]
    for Key in Keys:
        for Key_raw in Keys_raw:
            if Key[1]==Key_raw[1]:
                bundleDicts[runName][Key] = bundleDicts_raw[runName][Key_raw]

In [9]:
# check keys
dbRuns = list(resultDbs.keys())
bd_keys = list(bundleDicts[dbRuns[0]].keys())
print(bd_keys)

[(1, 'Nqso_i')]


In [10]:
iKey = (1, 'Nqso_i')

## Process the metric results for each OpSim and save the results in a table.

Note that in general, the rank will be from worst (0%) to best (100%)

In [11]:
def get_Nqso(Key, bds):
    runs = list(bds.keys())
    Nqso = np.zeros(len(runs))
    for k, run in enumerate(runs):
        mb  = bds[run][Key]
        mask = mb.metricValues.mask
        area_sq_deg = (mb.slicer.pixArea * u.sr).to(u.deg**2).value
        Nqso[k] = np.sum(mb.metricValues[~mask]*area_sq_deg)
    return Nqso

In [12]:
Nqso_i      = get_Nqso(iKey, bundleDicts)
i_rank_sort = np.argsort(Nqso_i)
i_areas     = get_opsim_areas(iKey, bundleDicts)
print(Nqso_i[i_rank_sort[0]], Nqso_i[i_rank_sort[-1]])

12022297.684583794 17635221.035760183


In [13]:
fname = "Table_Nqso_i.txt"
tab_file = open(fname,"w")
for k,run in enumerate(list(bundleDicts.keys())):
    tab_file.write("{0:50s}".format(run))
    
    i_rank_percent = (np.argwhere(k==i_rank_sort)[0,0]+1)/len(Nqso_i) * 100.
    tab_file.write(" {0:10.2f} {1:5.1f}".format(Nqso_i[k], i_rank_percent))
    tab_file.write(" {0:10.0f}".format(i_areas[k]))
                   
    tab_file.write("\n")
tab_file.close()